# Linear Regression

* [강의출처](https://www.youtube.com/watch?v=1JT8KhvymmY&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=6)
* [코드출처](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-04_1_multivariable_linear_regression.ipynb)